## Задача двухклассовой классификации. 

Столь малый объем обучающей выборки не позволяет нам обучить нейросеть с нуля. Я использовал предобученную модель ResNet50 (без последнего полносвязного слоя, делающего предсказания), и в процессе обучения ее веса были зафиксированы. Последний слой в моей модели составляют две softmax функции, так как число классов в нашей задаче 2. 

Обучающую выборку я предварительно аугментировал, увеличив ее объем в 24 раза. Трансформации: повороты на 90 градусов, зеркальное отражение + каждая картинка осветлена и затемнена с увеличенным контрастом 
Количество эпох сильно влияло на accuracy на тестовой выборке. Наилучшего результата я добился на трех эпохах.

Далее начиналось переобучение, так как accuracy на KAGGLE снижалось. Хорошую валидационную выборку из обучающей выборки создать не удалось, так как объем выборки и так очень маленький, к тому же в ней были повторяющиеся тарелки, и не было многих тарелок, содержащихся в тесте, поэтому смысла в ее создании я не увидел.

<img src="augmentation/cleaned/cleaned_0005.jpg1.jpg"><img src="augmentation/cleaned/cleaned_0005.jpg2.jpg"><img src="augmentation/cleaned/cleaned_0005.jpg3.jpg">

## Импорт необходимых библиотек

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.models import Sequential
from keras.applications import ResNet50
from keras.applications.imagenet_utils import preprocess_input
import pandas as pd
import numpy as np
import os

path = os.getcwd()

Using TensorFlow backend.


## Создание объектов ImageDataGenerator для обучающей и тестовой выборок, упрощающих input картинок в модель.

In [2]:
datagen = ImageDataGenerator(preprocessing_function = preprocess_input) 

train_generator = datagen.flow_from_directory(os.path.join(path, 'augmentation'), 
                                        target_size = (341, 256),
                                       batch_size = 32,
                                       class_mode = 'categorical')

test_generator = datagen.flow_from_directory(os.path.join(path, 'test'), 
                                        target_size = (341, 256),
                                       batch_size = 33,
                                       class_mode = 'categorical',
                                      shuffle = False)

Found 960 images belonging to 2 classes.
Found 660 images belonging to 1 classes.


## Загружаем из файла веса предобученной модели ResNet50

In [3]:
resnet = ResNet50(include_top = False, 
                   weights = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  pooling = 'avg')

Instructions for updating:
Colocations handled automatically by placer.


## Добавляем к ResNet связный слой из двух softmax функций.

In [4]:
model = Sequential()
model.add(resnet)
model.add(Dense(2, activation = 'softmax'))
model.layers[0].trainable = False
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics=['accuracy'])

## Обучение модели. Количество эпох по-разному отражается на финальном качестве модели. Наилучшего результата (0.979) модель достигала после обучения на 3 эпохах.

In [5]:
model.fit_generator(train_generator, steps_per_epoch = 30, epochs = 3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
30/30 [==============================] - 331s 11s/step - loss: 0.3226 - acc: 0.8896
Epoch 2/3
30/30 [==============================] - 327s 11s/step - loss: 0.1118 - acc: 0.9896
Epoch 3/3
30/30 [==============================] - 309s 10s/step - loss: 0.0779 - acc: 0.9948


## Делаем предсказания и выводим результаты в csv таблицу

In [6]:
predict = model.predict_generator(test_generator, steps = 20).argmax(axis=-1)

In [8]:
aydi = []
labels = []
for i in range(0, 660):
    aydi.append(str(i).zfill(4))
    if predict[i] == 1: 
        labels.append('dirty')
    else:
        labels.append('cleaned')
pd.DataFrame({'id': aydi, 'label': labels}).to_csv("out.csv", index = False, index_label = False)